In [1]:
# dependencies go here
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float 
from config import pw, gkey
import os

## set up database

We set up the `minneapolis_housing` database and related tables in Postgresql, using SQLalchemy:

* `neighborhood`
* `home_value`
* `crime`

Code for setting up the tables directly in Postgres can be found in the queries.sql file accompanying this notebook.

In [2]:
conn = f"postgres:{pw}@localhost:5432/minneapolis_housing"
engine = create_engine(f'postgresql://{conn}')

In [3]:
Base = declarative_base()

In [4]:
# Drop tables if they exist
# Neighborhood.__table__.drop(engine)
# Crime.__table__.drop(engine)
# Home_Value.__table__.drop(engine)
# OR 
# Base.metadata.drop_all(engine)

In [5]:
# Create base for later creating neighborhood table
class Neighborhood(Base):
    __tablename__ = 'neighborhood'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    neighborhood = Column(String(255))
    population_2010 = Column(Integer)

In [6]:
# Create base for later creating crime table
class Crime(Base):
    __tablename__ = 'crime'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    neighborhood_id = Column(Integer)
    crime_description = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)

In [7]:
class Home_Value(Base):
    __tablename__ = 'home_value'
    __table_args__ = {'extend_existing': True} 
    id = Column(Integer, primary_key=True)
    neighborhood_id = Column(Integer)
    property_type = Column(String(255))
    num_bedrooms = Column(Integer)
    sq_footage_house = Column(Integer)
    sq_footage_parcel = Column(Integer)
    address = Column(String(255))
    landuse = Column(String(255))
    value_total = Column(Integer)
    latitude = Column(Float)
    longitude = Column(Float)

## load data
### Neighborhood datasets

In [8]:
# Load Minneapolis city dataset for neighborhoods
n_path = os.path.join('.', 'data', 'Minneapolis_Neighborhoods.csv')
neighborhood = pd.read_csv(n_path)
neighborhood_df = neighborhood[['FID', 'BDNAME']].copy()
neighborhood_df.rename(columns={
    'FID': 'id',
    'BDNAME': 'neighborhood'
}, inplace=True)

'''
NOTES:
Noticed there wasn't a codebook that explained what the cryptic column names mean, so used FID based on the fact
that it is a unique ID.
'''

"\nNOTES:\nNoticed there wasn't a codebook that explained what the cryptic column names mean, so used FID based on the fact\nthat it is a unique ID.\n"

In [9]:
# Load Minneapolis neighborhood census dataset 
census_path = os.path.join('.', 'data', 'census_2010.xls')
census = pd.read_excel(census_path, header=None)
census_df = census.iloc[6:, :2]
census_df.rename(columns={
    0: 'neighborhood',
    1: 'population_2010'
}, inplace=True)

In [10]:
# merge the dfs together into one neighborhood df
neighborhoods = neighborhood_df.merge(census_df, how='outer', on='neighborhood')

In [11]:
# Check dataframe for null values
mask = pd.isnull(neighborhoods.id)
print(neighborhoods[mask])

mask = pd.isnull(neighborhoods.population_2010)
print(neighborhoods[mask])

# No population data for South Uptown (44) or Kenwood (74)
# No ids for CARAG (87) or Kenwood (88)

    id neighborhood population_2010
87 NaN        CARAG            5647
88 NaN     Kenwood             1414
89 NaN        Total          382578
      id  neighborhood population_2010
43  44.0  South Uptown             NaN
73  74.0       Kenwood             NaN


In [12]:
# deal with null values 

# Kenwood (74) should be updated with Kenwood (88)'s pop data 
neighborhoods.loc[73, 'population_2010'] = neighborhoods.loc[88, 'population_2010']
neighborhoods.tail(20)

# create id for CARAG (87)
neighborhoods.loc[87, 'id'] = '88'

# Drop rows 88, 89
neighborhoods.drop([88,89], inplace=True)

In [13]:
# Jenna start
#Load Assessors Parcel Data 2019 CSV & Create dataframe
csv_file = "data/Assessors_Parcel_Data_2019.csv"
assessors_df = pd.read_csv(csv_file)
assessors_df.head()

/Users/stacy/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (3,22,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,APN,ANUMBER,ANUMBERSUF,ST_PRE_DIR,ST_NAME,ST_TYPE,ST_POST_DIR,AUNIT,ZIPCODE,FORMATTED_ADDRESS,...,GARAGE_PRESENT,PRIMARYHEATING,CONSTRUCTIONTYPE,EXTERIORTYPE,ROOF,TOTAL_UNITS,FIREPLACES,BATHROOMS,BEDROOMS,ESRI_OID
0,102824420084,3536.0,NaN,NaN,25TH,Ave,S,NaN,55406-2536,3536 25TH Ave S,...,Yes,Forced Air,Wood Frame,Other,Gable,1.0,0.0,1.0,1.0,1001
1,102824420085,3540.0,NaN,NaN,25TH,Ave,S,NaN,55406-2536,3540 25TH Ave S,...,Yes,Forced Air,Wood Frame,Stucco,Gable,1.0,0.0,1.0,2.0,1002
2,102824420086,3544.0,NaN,NaN,25TH,Ave,S,NaN,55406-2536,3544 25TH Ave S,...,Yes,Forced Air,Wood Frame,Stucco,Gable,1.0,0.0,1.0,2.0,1003
3,102824420087,3546.0,NaN,NaN,25TH,Ave,S,NaN,55406-2536,3546 25TH Ave S,...,Yes,Forced Air,Wood Frame,Metal/Vinyl,Gable,1.0,0.0,1.0,2.0,1004
4,102824420088,3552.0,NaN,NaN,25TH,Ave,S,NaN,55406-2536,3552 25TH Ave S,...,Yes,Forced Air,Wood Frame,Stucco,Gable,1.0,0.0,1.0,2.0,1005


In [14]:
#Create a new dataframe with select columns
assessors_df = assessors_df[['NEIGHBORHOOD', 
                                    'FORMATTED_ADDRESS',
                                   'PROPERTY_TYPE',
                                   'LANDUSE',
                                   'TOTALVALUE',
                                   'BELOWGROUNDAREA',
                                   'ABOVEGROUNDAREA',
                                   'BEDROOMS',
                                   'PARCEL_AREA_SQFT',
                                   'X',
                                   'Y']]
assessors_df.head()

,NEIGHBORHOOD,FORMATTED_ADDRESS,PROPERTY_TYPE,LANDUSE,TOTALVALUE,BELOWGROUNDAREA,ABOVEGROUNDAREA,BEDROOMS,PARCEL_AREA_SQFT,X,Y
0,CORCORAN,3536 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,153000,256.0,712.0,1.0,4880,538085.657223,153746.075591
1,CORCORAN,3540 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,242000,853.0,1353.0,2.0,4880,538085.928915,153706.144539
2,CORCORAN,3544 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,192500,748.0,748.0,2.0,4880,538086.220841,153664.877318
3,CORCORAN,3546 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,210000,858.0,858.0,2.0,4880,538086.796681,153625.023665
4,CORCORAN,3552 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,214500,572.0,887.0,2.0,4880,538087.258047,153585.649166


In [15]:
#Rename columns
assessors_parcel_df = assessors_df.rename(columns={"NEIGHBORHOOD":"neighborhood",
                                                   "FORMATTED_ADDRESS": "address",
                                                    "PROPERTY_TYPE": "property_type",
                                                    "LANDUSE": "landuse",
                                                    "TOTALVALUE":"value_total",
                                                    "BELOWGROUNDAREA": "below_grade_sq_footage",
                                                    "ABOVEGROUNDAREA": "above_grade_sq_footage",
                                                    "BEDROOMS": "num_bedrooms",
                                                    "PARCEL_AREA_SQFT": "sq_footage_parcel",
                                                    "X": "x",
                                                    "Y": "y"})

In [16]:
#Filter out landuse categories that are not residential
residential_assessors_df = assessors_parcel_df[(assessors_parcel_df.landuse =='SINGLE-FAMILY DETACHED DWELLING')
                            |(assessors_parcel_df.landuse =='SINGLE-FAMILY ATTACHED DWELLING')
                            |(assessors_parcel_df.landuse =='MULTI-FAMILY RESIDENTIAL')
                            |(assessors_parcel_df.landuse =='MULTI-FAMILY APARTMENT')]

In [17]:
#Filter out property types categories that are not residential
residential_df = residential_assessors_df[(residential_assessors_df.property_type =='Residential')
                |(residential_assessors_df.property_type =='Condominium')
                |(residential_assessors_df.property_type =='Double Bungalow')
                |(residential_assessors_df.property_type =='Apartment')
                |(residential_assessors_df.property_type =='Cooperative')
                |(residential_assessors_df.property_type =='Townhouse')
                |(residential_assessors_df.property_type =='Triplex')
                |(residential_assessors_df.property_type =='Residential - Zero Lot Line- DB')
                |(residential_assessors_df.property_type =='Residential Lake Shore')]                

residential_df.head()

,neighborhood,address,property_type,landuse,value_total,below_grade_sq_footage,above_grade_sq_footage,num_bedrooms,sq_footage_parcel,x,y
0,CORCORAN,3536 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,153000,256.0,712.0,1.0,4880,538085.657223,153746.075591
1,CORCORAN,3540 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,242000,853.0,1353.0,2.0,4880,538085.928915,153706.144539
2,CORCORAN,3544 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,192500,748.0,748.0,2.0,4880,538086.220841,153664.877318
3,CORCORAN,3546 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,210000,858.0,858.0,2.0,4880,538086.796681,153625.023665
4,CORCORAN,3552 25TH Ave S,Residential,SINGLE-FAMILY DETACHED DWELLING,214500,572.0,887.0,2.0,4880,538087.258047,153585.649166


In [20]:
residential_df.tail()

,neighborhood,address,property_type,landuse,value_total,below_grade_sq_footage,above_grade_sq_footage,num_bedrooms,sq_footage_parcel,x,y
129794,STEVENS SQ/LORING HGTS,1901 Stevens Ave #002,Condominium,SINGLE-FAMILY ATTACHED DWELLING,155000,NaN,978.0,2.0,6400,528060.219494,162987.160895
129795,STEVENS SQ/LORING HGTS,1901 Stevens Ave #101,Condominium,SINGLE-FAMILY ATTACHED DWELLING,117500,NaN,507.0,1.0,6400,528060.219494,162987.160895
129796,STEVENS SQ/LORING HGTS,1901 Stevens Ave #102,Condominium,SINGLE-FAMILY ATTACHED DWELLING,117500,NaN,507.0,1.0,6400,528060.219494,162987.160895
129797,STEVENS SQ/LORING HGTS,1901 Stevens Ave #103,Condominium,SINGLE-FAMILY ATTACHED DWELLING,120000,NaN,520.0,1.0,6400,528060.219494,162987.160895
129798,STEVENS SQ/LORING HGTS,1901 Stevens Ave #104,Condominium,SINGLE-FAMILY ATTACHED DWELLING,120000,NaN,520.0,1.0,6400,528060.219494,162987.160895


In [21]:
# Get lat/lng data based on address - planned to do this but it is cost prohibitive given the $0 budget for this
# project and the cost of querying the Google Geocoding API for ~130,000 addresses. 

In [ ]:
# Jenna end

In [ ]:
# Katrina start
#Loading data for police incidents and creating a dataframe for the data
csv_file = "data/Police_Incidents_2019.csv"
police_incidents_df = pd.read_csv(csv_file)
police_incidents_df.head()

In [ ]:
# Katrina end
#Create new dataframe with certain columns
incidents_df = police_incidents_df[['description',
                                    'neighborhood',
                                    'X',
                                    'Y']]
incidents_df

## transforming data

Things to transform all the data
1. Lower-case the neighborhood names on both dfs
2. Merge dfs on neighborhood names (into new crime df)
3. Drop neighborhood name from new crime df
4. Do same merge and neighborhood drop on assessor df

In [ ]:
# Stacy start

In [ ]:
neighborhoods.neighborhood = neighborhoods.neighborhood.str.lower()

In [ ]:
# Stacy end

In [ ]:
# Jenna start

In [ ]:
# Jenna end

In [ ]:
# Katrina start

In [ ]:
# Katrina end

## adding to postgres

In [ ]:
# Stacy start

In [ ]:
# load neighborhoods df into database table
neighborhoods.to_sql(name='neighborhood', con=engine, if_exists='replace', index=False)

In [ ]:
# Check that data is loaded into postgres
pd.read_sql_query('select * from neighborhood', con=engine).head()

In [ ]:
# Stacy end

In [ ]:
## Pulling it all together in an SQLalchemy query/pandas df for aggregate table